In [1]:
# !pip install transformers
# !pip install sentencepiece
# !pip install accelerate -U
# !pip install sacrebleu
# !pip install unbabel-comet

In [2]:
# !pip install datasets

In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from sacrebleu.metrics import BLEU, CHRF
from comet import download_model, load_from_checkpoint
from datasets import load_dataset
import time
import warnings
warnings.filterwarnings("ignore")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
dataset = load_dataset("wmt17", "de-en", split="validation")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/5906184 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3004 [00:00<?, ? examples/s]

Dataset wmt17 downloaded and prepared to /root/.cache/huggingface/datasets/wmt17/de-en/1.0.0/626ef4b4893062acbd74367a303a029b0a61c6a093453dd1e55477155d7b27c2. Subsequent calls will reuse this data.


In [5]:
dataset['translation'][0]['de']


'Obama empfängt Netanyahu'

In [6]:
en_Sentences = [ i['en'] for i in dataset['translation'] ]
de_Sentences =[ i['de'] for i in dataset['translation'] ]

print(len(en_Sentences))
print(len(de_Sentences))

2999
2999


In [7]:
t0 = time.time()
modelName = "facebook/mbart-large-50-many-to-many-mmt" # "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(modelName)
tokenizer = MBart50TokenizerFast.from_pretrained(modelName)
print(f'time in minutes is : {(time.time() - t0)/60} m')

time in minutes is : 0.7710665265719095 m


In [8]:
print(tokenizer.vocab_size)

250054


## Generating Translation

In [9]:
%%time
t0 = time.time()

generated_translations = []
i=0
for english_sentence in en_Sentences:
    if (i%100==0):
        print(i)
    tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(english_sentence, return_tensors="pt", max_length=1024, truncation=True)
    generated_tokens = model.generate(
        **encoded_en,
        forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"]
    )
    gen_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    generated_translations.append(gen_text)
    i+=1
print(f'time in minutes is : {(time.time() - t0)/60} m')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
time in minutes is : 529.8735136389732 m
CPU times: user 17h 6min 10s, sys: 1min 35s, total: 17h 7min 46s
Wall time: 8h 49min 52s


In [3]:
# generated_translations

In [11]:
generated_translationss = [text[0] for text in generated_translations]


In [12]:
len(generated_translationss)

2999

In [13]:
ref = [i for i in de_Sentences]

In [14]:
len(ref)

2999

## BLEU and CHRF score

In [15]:

bleu = BLEU()

print(f'BLEU score is : {bleu.corpus_score(generated_translationss, [ref])}')

chrf = CHRF()
print(f'chrF2 score is :{chrf.corpus_score(generated_translationss, [ref])}')

BLEU score is : BLEU = 34.33 64.9/40.3/27.5/19.3 (BP = 1.000 ratio = 1.001 hyp_len = 62738 ref_len = 62669)
chrF2 score is :chrF2 = 61.49


## Comet

In [16]:

comt_model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comt_model_path)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
data = [
    {
        "src": en_Sentences,
        "mt":  generated_translationss,
        "ref": [ref]
    }
]

In [18]:
# Calculate COMET score
comet_model_output = comet_model.predict(data, batch_size=8, gpus=1)
print(comet_model_output)

Predicting: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:10<00:00, 10.67s/it]


Prediction([('scores', [0.805546760559082]), ('system_score', 0.805546760559082)])
